In [66]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

# openai.api_key = "*************"
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [38]:
text[265: 268]

'\n \n'

In [52]:
matches = re.finditer(pattern, text)
for match in matches:
    print(match.group(1),match.end)
#     print(match.start())

SOUTHERN DISTRICT OF NEW YORK  <built-in method end of re.Match object at 0x000002926D0CE330>
  <built-in method end of re.Match object at 0x000002926D0CEA30>
  <built-in method end of re.Match object at 0x000002926D0CE330>
  <built-in method end of re.Match object at 0x000002926D0CE6B0>
    <built-in method end of re.Match object at 0x000002926D0CE330>
  <built-in method end of re.Match object at 0x000002926D0CE6B0>
 JURY TRIAL DEMANDED     <built-in method end of re.Match object at 0x000002926D0CEA30>
COMPLAINT  <built-in method end of re.Match object at 0x000002926D0CE6B0>
SUMMARY  <built-in method end of re.Match object at 0x000002926D0CEA30>
JURISDICTION AND VENUE  <built-in method end of re.Match object at 0x000002926D0CE6B0>
DEFENDANTS  <built-in method end of re.Match object at 0x000002926D0CEA30>
BACKGROUND ON SECU RITIES OFFERINGS  <built-in method end of re.Match object at 0x000002926D0CE6B0>
BACKGROUND ON CRYPTO ASSETS  <built-in method end of re.Match object at 0x000002926

In [56]:
import requests
import PyPDF2
import re
import io

# Download the PDF file from the URL
url = 'https://www.sec.gov/litigation/complaints/2023/comp-pr2023-59.pdf'
response = requests.get(url)

# Extract the text from the PDF file
pdf_file = PyPDF2.PdfReader(io.BytesIO(response.content))
text = ''
for page_num in range(len(pdf_file.pages)):
    page = pdf_file.pages[page_num]
    text += page.extract_text()

# Split the text into sections based on the headings
sections = {}
pattern = r'\n([A-Z ]+)\n'
matches = re.finditer(pattern, text)
start = 0
# Add the last section
# sections[''] = text[start:].strip()
for match in matches:
    end = match.start()
    heading = match.group(1)
    sections[heading] = text[start:end].strip()
    start = end

# Add the last section
sections[''] = text[end:].strip()

sections

{'SOUTHERN DISTRICT OF NEW YORK ': 'UNITED STATES DISTRICT COURT',
 ' ': 'SIXTH CLAIM FOR RELIEF \nAiding and Abetting Violation of S ection 17(b) of the Securities Act \n(Against Sun, the Tron Foundation, the BitTorrent Foundation, and Rainberry)',
 '   ': 'Defendants.',
 ' JURY TRIAL DEMANDED    ': 'Case No. 23-2433',
 'COMPLAINT ': 'JURY TRIAL DEMANDED',
 'SUMMARY ': 'COMPLAINT \nPlaintiff Securities and Exchange Commissi on (the “SEC” or “Com mission”) alleges the \nfollowing:',
 'JURISDICTION AND VENUE ': 'SUMMARY \n1. This action arises from Defendant Jus tin Sun’s (“Sun”) or chestration of the \nunregistered offer and sale, manipulative trading, and unlawful touting of crypto asset securities. \n2. Working through several companies that  he owns and controls—including \nDefendants Tron Foundation Limited (“Tron Foundation”), BitTorrent Foundation Ltd. \n(“BitTorrent Foundation”), and Rainberry, Inc. (“ Rainberry,” and collectively with Sun, the Tron \nFoundation, and the BitTorr

In [31]:
sections.values()

dict_values(['SOUTHERN DISTRICT OF NEW YORK \nSECURITIES AND EX CHANGE COMMISSION, \n \nPlaintiff, \n \nv.  \nJUSTIN SUN, TRON FOUNDATION LIMITED, \nBITTORRENT FOUNDATION LTD., \nRAINBERRY, INC., AUSTIN MAHONE, and \nDEANDRE CORTEZ WAY, \n \nDefendants. \n   \n \n \n \nCase No. 23-2433 \n JURY TRIAL DEMANDED    \n \nCOMPLAINT \nPlaintiff Securities and Exchange Commissi on (the “SEC” or “Com mission”) alleges the \nfollowing: \nSUMMARY \n1. This action arises from Defendant Jus tin Sun’s (“Sun”) or chestration of the \nunregistered offer and sale, manipulative trading, and unlawful touting of crypto asset securities. \n2. Working through several companies that  he owns and controls—including \nDefendants Tron Foundation Limited (“Tron Foundation”), BitTorrent Foundation Ltd. \n(“BitTorrent Foundation”), and Rainberry, Inc. (“ Rainberry,” and collectively with Sun, the Tron \nFoundation, and the BitTorrent Foundation, the “S un Defendants”)—Sun e ngineered the offer \nand sale of two cr

In [57]:
pdf_df = pd.DataFrame(sections.items(), columns = ['topic', 'content'])
pdf_df['topic'] = pdf_df['topic'].shift(1)
pdf_df

,topic,content
0,None,UNITED STATES DISTRICT COURT
1,SOUTHERN DISTRICT OF NEW YORK,SIXTH CLAIM FOR RELIEF \nAiding and Abetting V...
2,,Defendants.
3,,Case No. 23-2433
4,JURY TRIAL DEMANDED,JURY TRIAL DEMANDED
5,COMPLAINT,COMPLAINT \nPlaintiff Securities and Exchange ...
6,SUMMARY,SUMMARY \n1. This action arises from Defendant...
7,JURISDICTION AND VENUE,JURISDICTION AND VENUE \n10. The Court has sub...
8,DEFENDANTS,DEFENDANTS \n12. Defendant Justin Sun (a/k/a ...
9,BACKGROUND ON SECU RITIES OFFERINGS,BACKGROUND ON SECU RITIES OFFERINGS \n18. Cong...


In [92]:
pdf_df = pdf_df[pdf_df['content'].apply(lambda x: len(x))>=100].reset_index(inplace=True, drop=True)
pdf_df

,topic,content
0,SOUTHERN DISTRICT OF NEW YORK,SIXTH CLAIM FOR RELIEF \nAiding and Abetting V...
1,COMPLAINT,COMPLAINT \nPlaintiff Securities and Exchange ...
2,SUMMARY,SUMMARY \n1. This action arises from Defendant...
3,JURISDICTION AND VENUE,JURISDICTION AND VENUE \n10. The Court has sub...
4,DEFENDANTS,DEFENDANTS \n12. Defendant Justin Sun (a/k/a ...
5,BACKGROUND ON SECU RITIES OFFERINGS,BACKGROUND ON SECU RITIES OFFERINGS \n18. Cong...
6,BACKGROUND ON CRYPTO ASSETS,BACKGROUND ON CRYPTO ASSETS \n22. The term “cr...
7,FACTS,2 “Staking” concerns the “proof of stake” val...
8,,"176. As illustrated in the tables above, the t..."
9,FIRST CLAIM FOR RELIEF,191. Paragraphs 1 through 189 above are re-a l...


In [5]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }


In [94]:
document_embeddings = compute_doc_embeddings(pdf_df)

In [59]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

0 : [-0.007944014854729176, -0.013504824601113796, 0.003222666447982192, -0.01846482791006565, 0.0011732442071661353]... (1536 entries)


In [62]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [95]:
order_document_sections_by_query_similarity("Who are the defendants in this case?", document_embeddings)[:5]

[(0.8080617896888086, 4),
 (0.8032874582562652, 0),
 (0.796536446363935, 3),
 (0.7926742822248016, 14),
 (0.7913491915430325, 9)]

In [98]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"


'Context separator contains 3 tokens'

In [81]:
pdf_df.loc[8]['content']

'DEFENDANTS \n12. Defendant Justin Sun  (a/k/a Yuchen Sun), age 32, is a Chinese national and \ncrypto asset entrepreneur.  Sun attended colleg e in China and received a post-graduate degree \nfrom the University of Pennsylvania.  Sun is cu rrently the Permanent Representative of Grenada \nto the World Trade Organization and is believed to be living in Singapore and/or Hong Kong.  At \nall relevant times, Sun acted through, on behalf of, and for th e benefit of the Tron Foundation, \nthe BitTorrent Foundation, and Rainberry.  \n13. Defendant Tron Foundation Limited  was founded by Sun in 2017 as a \nSingapore Public Company Limited by Guarantee.  The Tron Foundation is one of several alter-\nego entities organized by Sun through which he conducted the TRX and BTT offerings and \nmanaged the development and marketing of the Tron ecosystem.  From 2017 to the present, Sun \nhas owned and controlled the Tron Foundation.    \n14. Defendant BitTorrent Foundation Ltd.  (sometimes d/b/a BitTor

In [100]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += len(encoding.encode(document_section['content'])) + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"


In [101]:
prompt = construct_prompt(
    "Who are the defendants in this case?",
#     "Who are the involved parties in this case?",
    document_embeddings,
    pdf_df
)

print("===\n", prompt)

Selected 3 document sections:
4
0
3
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* DEFENDANTS  12. Defendant Justin Sun  (a/k/a Yuchen Sun), age 32, is a Chinese national and  crypto asset entrepreneur.  Sun attended colleg e in China and received a post-graduate degree  from the University of Pennsylvania.  Sun is cu rrently the Permanent Representative of Grenada  to the World Trade Organization and is believed to be living in Singapore and/or Hong Kong.  At  all relevant times, Sun acted through, on behalf of, and for th e benefit of the Tron Foundation,  the BitTorrent Foundation, and Rainberry.   13. Defendant Tron Foundation Limited  was founded by Sun in 2017 as a  Singapore Public Company Limited by Guarantee.  The Tron Foundation is one of several alter- ego entities organized by Sun through which he conducted the TRX and BTT offerings and  managed the dev

In [102]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [103]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [105]:
answer_query_with_context("Who are the defendants in this case?", pdf_df, document_embeddings)

Selected 3 document sections:
4
0
3


'The defendants in this case are Justin Sun (a/k/a Yuchen Sun), the Tron Foundation Limited, the BitTorrent Foundation Ltd., Rainberry, Inc., Austin Mahone, and DeAndre Cortez Way (a/k/a Soulja Boy).'